In [17]:
import numpy as np
import itertools
import json
import os
from tqdm import tqdm

def load_decks(path):
    """Load deck data from .npy file and convert to binary strings."""
    data = np.load(path, allow_pickle = True)
    decks = data[:, 1]
    decks.tolist()
   
    #decks = decks[:10000] #for testing making it shorter
    
    return decks

def score_deck(deck: str, seq1: str, seq2: str) -> tuple[int, int, int, int]:
    """Score a single deck for both players."""
    p1_cards = p2_cards = 0
    p1_tricks = p2_tricks = 0
    pile = 2
    i = 0
    
    while i < len(deck) - 2:
        pile += 1
        current_sequence = deck[i:i+3]
        if current_sequence == seq1:
            p1_cards += pile
            p1_tricks += 1
            pile = 2
            i += 3
        elif current_sequence == seq2:
            p2_cards += pile
            p2_tricks += 1
            pile = 2
            i += 3
        else:
            i += 1
    
    return p1_cards, p2_cards, p1_tricks, p2_tricks

def calculate_winner(p1_cards: int,
                     p2_cards: int,
                     p1_tricks: int,
                     p2_tricks: int):
        '''Given the number of cards and tricks for each player, calculate who wins for cards and tricks, as well as draws for cards and tricks.
            If player one wins, the winner is set to 0. If player 2 wins, the winner is set to 1.
            Also indicates if there was a draw.

        Arguments:
            p1_cards (int): number of cards player 1 won
            p2_cards (int): number of cards player 2 won
            p1_tricks (int): number of tricks player 1 won
            p2_tricks (int): number of tricks player 2 won
        
        Output:
            cards_winner (int): specifies who won based on cards
            cards_draw (int): 1 if a draw occurred, 0 otherwise
            tricks_winner (int): specifies who won based on tricks
            tricks_draw (int): 1 if a draw occured, 0 otherwise'''
        cards_winner = 0
        cards_draw = 0
        tricks_winner = 0
        tricks_draw = 0

        # if p2 wins set winner to 1, otherwise it is 0 (including draws).
        # if there is a draw, set draw counter to 1
        if p1_cards > p2_cards:
            cards_winner = 1
        elif p1_cards == p2_cards:
            cards_draw = 1
        if p1_tricks > p2_tricks:
            tricks_winner = 1
        elif p1_tricks == p2_tricks:
            tricks_draw = 1
        return cards_winner, cards_draw, tricks_winner, tricks_draw


def generate_unique_combinations(sequences):
    """
    Generate unique combinations from the given sequences, accounting for inversions.
    
    Inputs:
    - sequences (list of str): List of binary sequences to generate combinations from.
    Returns:
    - set: Set of unique sequence pairs (accounting for inversions).
    """
    # Step 1: Generate all unique pairs of sequences without repetition
    pairs = set(itertools.combinations(sequences, 2))
    
    # Step 2: Use a set to store unique combinations considering inversion
    unique_combinations = set()
    
    for pair in pairs:
        bin1, bin2 = pair
        
        # Create the inverted version of the pair
        inverted_pair = ('{0:03b}'.format(~int(bin1, 2) & 7), '{0:03b}'.format(~int(bin2, 2) & 7))
        
        # Sort both the pair and the inverted pair to standardize
        sorted_pair = tuple(sorted(pair))
        sorted_inverted_pair = tuple(sorted(inverted_pair))
        
        # Add the pair only if its inverse or its swapped version isn't already present
        if sorted_inverted_pair not in unique_combinations:
            unique_combinations.add(sorted_pair)
    
    return unique_combinations


def process_all_decks(decks):
    """Process all decks and compute statistics for all sequence combinations."""
    sequences = ['000', '001', '010', '011', '100', '101', '110', '111']
    n_sequences = len(sequences)
    
    #unique_combinations = generate_unique_combinations(sequences) #gives us 16 combos to use
    
    # Initialize result arrays
    cards_wins = np.zeros((n_sequences, n_sequences))
    tricks_wins = np.zeros((n_sequences, n_sequences))
    cards_ties = np.zeros((n_sequences, n_sequences))
    tricks_ties = np.zeros((n_sequences, n_sequences))
    
    # Create sequence index mapping
    seq_to_idx = {seq: idx for idx, seq in enumerate(sequences)}
    
    total_decks = len(decks)
    
    # Process each deck
    for deck in tqdm(decks, desc="Processing decks"):
        for seq1 in sequences: #using unique_combinations would generate half the gird, 28 pairs
            for seq2 in sequences: 
                if seq1 != seq2: 
                    idx1 = seq_to_idx[seq1]  # Getting index of sequence 1
                    idx2 = seq_to_idx[seq2]  # Getting index of sequence 2
                
                    p1_cards, p2_cards, p1_tricks, p2_tricks = score_deck(deck, seq1, seq2)
                    cards_winner, cards_draw, tricks_winner, tricks_draw = calculate_winner(p1_cards, p2_cards, p1_tricks, p2_tricks)
            
            # Update win counts
            
           # if cards_winner == 1: #player 1 wins 
            #    cards_wins[idx1][idx2] += 1
           # elif cards_draw:
             #   cards_ties[idx1][idx2] += 1
                
           # if tricks_winner == 1:
            #    tricks_wins[idx1][idx2] += 1
           # elif tricks_draw:
               # tricks_ties[idx1][idx2] += 1
            
                    if cards_draw:
                        cards_ties[idx1][idx2] += 1
                    elif cards_winner == 1:  # Player 1 wins
                        cards_wins[idx1][idx2] += 1
            
                    if tricks_draw:
                        tricks_ties[idx1][idx2] += 1
                    elif tricks_winner == 1:  # Player 1 wins
                        tricks_wins[idx1][idx2] += 1
            
            
            

    # Convert to probabilities
    cards_wins = cards_wins / total_decks
    tricks_wins = tricks_wins / total_decks
    cards_ties = cards_ties / total_decks
    tricks_ties = tricks_ties / total_decks
    
    return {
        'cards': cards_wins.tolist(),
        'tricks': tricks_wins.tolist(),
        'cards_ties': cards_ties.tolist(),
        'tricks_ties': tricks_ties.tolist(),
        'n': total_decks
    }

def process_and_save_results(input_path, output_folder='results'):
    """Process decks from input file and save results to output folder."""
    # Create output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)
    
    # Load and process decks
    print("Loading decks...")
    decks = load_decks(input_path)
    
    # Process all decks and get results
    print("Processing games...")
    results = process_all_decks(decks)
    
    # Save results
    output_path = os.path.join(output_folder, 'results.json')
    with open(output_path, 'w') as f:
        json.dump(results, f)
    
    return results

# Example usage
# process_and_save_results("path_to_your_npy_file.npy")

In [18]:
process_and_save_results('/Users/student/Desktop/fall 2024/Automation/Project Penney/penney_final_revisions/deck_data.npy')



Loading decks...
Processing games...


Processing decks: 100%|█████████████| 1000000/1000000 [13:02<00:00, 1278.62it/s]


{'cards': [[0.0,
   0.471181,
   0.277245,
   0.176197,
   9e-06,
   0.252084,
   0.009624,
   0.485966],
  [0.508744, 0.0, 0.863651, 0.955683, 0.001951, 0.918409, 0.478439, 0.988121],
  [0.706454, 0.123296, 0.0, 0.485003, 0.418689, 0.488421, 0.068325, 0.725582],
  [0.803865, 0.036044, 0.486251, 0.0, 0.484757, 0.558079, 0.997592, 0.999979],
  [0.999983, 0.997593, 0.559711, 0.486521, 0.0, 0.485953, 0.036419, 0.8033],
  [0.72634, 0.067817, 0.487961, 0.420119, 0.485389, 0.0, 0.123647, 0.705578],
  [0.987745, 0.476946, 0.91769, 0.001919, 0.955389, 0.863371, 0.0, 0.507734],
  [0.485146, 0.009331, 0.253137, 1.4e-05, 0.176676, 0.278278, 0.472292, 0.0]],
 'tricks': [[0.0,
   0.403947,
   0.211763,
   0.112831,
   0.001116,
   0.168263,
   0.008337,
   0.324818],
  [0.441419, 0.0, 0.802194, 0.88356, 0.026337, 0.793117, 0.410226, 0.967088],
  [0.663588, 0.115577, 0.0, 0.43066, 0.421744, 0.406848, 0.113285, 0.693034],
  [0.76947, 0.05138, 0.432875, 0.0, 0.431389, 0.442881, 0.935577, 0.994948],
  